# __Demo: New Tree of Thought Prompting with OpenAI__

This notebook demonstrates Tree of Thoughts prompting with strict logical validation.

In [1]:
import os
import openai

client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [5]:
river_crossing_prompt = """
Solve this puzzle using strict logical reasoning:

A farmer needs to transport a wolf, a goat, and a cabbage across a river.
Rules:
1. The boat can only carry one item (plus the farmer)
2. If wolf and goat are on the same bank WITHOUT the farmer: wolf eats goat
3. If goat and cabbage are on the same bank WITHOUT the farmer: goat eats cabbage
4. The farmer must make each crossing
5. The farmer should not be alone on either of the two banks.

Start with State 0:
Left bank: Wolf, Goat, Cabbage, Farmer
Right bank: (empty)

For each state, follow EXACTLY this format:

State [number]:
Left bank: [list items]
Right bank: [list items]
Farmer location: [Left/Right] bank

Safety Check:
1. Left bank check:
   - Is farmer present? [Yes/No]
   - If No, check:
     * Wolf and goat together? [Yes/No]
     * Goat and cabbage together? [Yes/No]

2. Right bank check:
   - Is farmer present? [Yes/No]
   - If No, check:
     * Wolf and goat together? [Yes/No]
     * Goat and cabbage together? [Yes/No]

State is UNSAFE if any bank without farmer has:
- Wolf and goat together, OR
- Goat and cabbage together
any action that would lead to an UNSAFE state should be rejected and other options should be considered.

If state is UNSAFE:
- Explain which rule is violated
- Return to previous state
- Try different move

If state is SAFE:
Possible moves:
1. [list each possible item farmer could take]
2. [or farmer could go alone]

Chosen move: [describe move]
Reasoning: [explain why this move was chosen]

STOP when all items reach the right bank.
Show your reasoning step by step.
"""

print("River Crossing Solution:")
print(get_completion(river_crossing_prompt))

River Crossing Solution:
State 0:
Left bank: Wolf, Goat, Cabbage, Farmer
Right bank: (empty)
Farmer location: Left bank

Safety Check:
Left bank check:
- Is farmer present? Yes
- Wolf and goat together? No
- Goat and cabbage together? No

Right bank check:
- Is farmer present? No
- Wolf and goat together? No
- Goat and cabbage together? No

State is SAFE

Possible moves:
1. Farmer and Wolf
2. Farmer and Goat
3. Farmer and Cabbage
4. Farmer alone

Chosen move: Farmer and Goat
Reasoning: Taking the goat first ensures that the wolf and goat are not left alone together on the left bank, which would violate rule 2.

State 1:
Left bank: Wolf, Cabbage, Farmer
Right bank: Goat
Farmer location: Right bank

Safety Check:
Left bank check:
- Is farmer present? Yes
- Wolf and goat together? No
- Goat and cabbage together? No

Right bank check:
- Is farmer present? Yes
- Wolf and goat together? No
- Goat and cabbage together? No

State is SAFE

Possible moves:
1. Farmer alone
2. Farmer and Wolf
3. F

### Why This Approach Is Better

1. **Explicit Safety Checking**:
   - Must check each bank separately
   - Must first check if farmer is present
   - Only check dangerous combinations if farmer is absent

2. **Clear State Representation**:
   - Shows exact contents of each bank
   - Shows farmer's location
   - No ambiguous terms

3. **Strict Move Validation**:
   - Must list all possible moves
   - Must explain choice
   - Must verify safety after move

4. **Proper Backtracking**:
   - Must identify specific rule violation
   - Must return to previous safe state
   - Must try different move